In [128]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [129]:
year = 2023
url = "http://pfref.com/pi/share/gL6Ax"
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

In [154]:
headers = [th.get_text(strip=True) for th in soup.findAll('tr')[1].findAll('th')]

# Desired columns (with "Tm" as the first header)
desired_columns = ["Tm", "G", "Att", "Yds", "TD"]

# Adjust headers to include only desired columns
adjusted_headers = [header for header in headers if header in desired_columns]

# Print the headers for verification
print("Adjusted Headers:", adjusted_headers)

Adjusted Headers: ['Tm', 'Att', 'Yds', 'TD']


In [155]:
rows = soup.findAll('tr')[2:]  # Skip header rows

player_stats = []
for row in rows:
    # The team name might be in the first <th> or <td> cell
    team_name = row.find('th').get_text(strip=True) if row.find('th') else row.find('td').get_text(strip=True)
    
    # Get the rest of the data cells (<td>)
    cells = [td.get_text(strip=True) for td in row.findAll('td')]
    
    # Combine team name with the rest of the data cells
    if len(cells) == len(adjusted_headers) - 1:
        player_stats.append([team_name] + cells)
    else:
        print(f"Skipping row: {cells} (Expected {len(adjusted_headers) - 1} data columns, got {len(cells)})")

In [157]:
stats = pd.DataFrame(player_stats, columns=adjusted_headers)

# Step 7: Display the first few rows of the DataFrame
print(stats.head())

                   Tm   Att    Yds    TD
0   Arizona Cardinals  25.8  119.2  0.82
1    Seattle Seahawks  24.0  106.8  1.12
2      Denver Broncos  22.6  118.4  0.71
3   Carolina Panthers  24.9  109.0  1.24
4  Indianapolis Colts  25.7  103.9  0.94


In [158]:
stats.to_csv('23_defense_rush_stats.csv')